In [9]:
import cv2
import numpy as np
import pytesseract
import re
import os
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'


In [14]:
def extract_transaction_info(text):
  # extract transaction information from the text
  transaction_info = {}

  # extract date
  date_pattern = r'DATE: (\d{2}/\d{2}/\d{4})|Date In (\d{2}/\d{2}/\d{2})'
  date_match = re.search(date_pattern, text)
  if date_match:
    if date_match.group(1):
      transaction_info["Date"] = date_match.group(1)
    else:
      transaction_info["Date"] = date_match.group(2)

  # extract total amount
  total_pattern = r'Net Total (\d+\,\d+)|Grand Total \(RS\) (\d+\.\d+)|TOTAL: Rs\.(\d+)'
#   total_pattern = r'NET TOTAL (\d+\.\d+)'
  total_match = re.search(total_pattern, text)
  if total_match:
    if total_match.group(1):
      transaction_info["Amount"] = total_match.group(1)
    elif total_match.group(2):
      transaction_info["Amount"] = float(total_match.group(2))
    else:
      transaction_info["Amount"] = float(total_match.group(3))

      
  return transaction_info

def extract_text_from_image(image_path):
    image = cv2.imread(image_path)
    # convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # gray = cv2.resize(gray, None, fx=.5, fy=.5)
    
    # apply Gaussian blurring to smooth out the image
    # thresh = cv2.GaussianBlur(gray, (7,7), 0)
    thresh = cv2.medianBlur(gray,9)

    # apply Otsu's thresholding to detect text regions
    # thresh = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 5, 11)
    # thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

    # apply dilation and erosion to remove noise
    kernel = np.ones((2,2), np.uint8)
    # kernel1 = np.ones((2,2), np.uint8)
    thresh = cv2.dilate(thresh, kernel, iterations=1)
    thresh = cv2.erode(thresh, kernel, iterations=1)

    cv2.imwrite("proceesimg.jpg",thresh)

    # set tesseract OCR to recognize handwritten text
    custom_oem_psm_config = r'--oem 3 --psm 6'

    # run tesseract OCR on the image
    text = pytesseract.image_to_string(thresh,config=custom_oem_psm_config)
    # print(text)
    return text


print(extract_transaction_info(extract_text_from_image( "./BillDataset/20221021_Grocery.jpg")))


{'Amount': '1,795'}


In [15]:
folder = "./bills/"

for filename in os.listdir(folder):
    print(filename)
    extext = extract_text_from_image(folder+filename)
    f = open(folder+filename+'.txt', "w")
    f.write(extext)
    f.close()


20200123_Other.jpg
20211031_HomeNeed.jpg
20211103_HomeNeed.jpg
20211103_HomeNeed_2.jpg
20211106_HomeNeed.jpg
20211108_HomeNeed.jpg
20211109_HomeNeed.jpg
20211119_HomeNeed.jpg
20220108_Other.jpg
20220210_Other.jpg
20220218_Other.jpg
20220328_BillPayment.jpg
20220410_Grocery.jpg
20220601_Food.jpg
20220622_BillPayment.jpg
20220717_Other.jpg
20220814_Food.jpg
20220821_Food.jpg
20220822_Grocery.jpg
20220824_BillPayment.jpg
20220915_Transport.jpg
20220918_Food.jpg
20220926_HomeNeed.jpg
20220926_HomeNeed_2.jpg
20220926_HomeNeed_3.jpg
20221001_Grocery.jpg
20221004_Food (2).jpg
20221004_Food.jpg
20221005_Grocery.jpg
20221015_Grocery.jpg
20221016_Grocery.jpg
20221021_Grocery.jpg
20221021_Transport.jpg
20221021_Transport_2.jpg
20221021_Transport_3.jpg
20221022_Food.jpg
20221022_Transport.jpg
20221022_Transport_2.jpg
20221022_Transport_3.jpg
20221022_Transport_4.jpg
20221023_Food.jpg
20221024_Grocery.jpg
20221025_BillPayment.jpg
20221027_Grocery.jpg
20221029_Grocery.jpg
20221030_Food.jpg
20221031_

image_path = "./BillDataset/20221021_Grocery.jpg"
image = cv2.imread(image_path)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# blur = cv2.medianBlur(gray, 5)


blur = cv2.GaussianBlur(gray, (13,13), 0)
# apply Otsu's thresholding to detect text regions
thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

# apply dilation and erosion to remove noise
kernel = np.ones((1,1), np.uint8)
# kernel1 = np.ones((2,2), np.uint8)
thresh = cv2.dilate(thresh, kernel, iterations=1)
thresh = cv2.erode(thresh, kernel, iterations=1)
cv2.imwrite("proceesimg.jpg",thresh)


#### create a python model for extract transaction information from a bank sms among others

- First, you will need to obtain a dataset of bank SMS messages that contain transaction information. You can either create this dataset yourself by manually collecting and annotating SMS messages, or you can try to find a pre-existing dataset online.

- Next, you will need to preprocess the dataset by cleaning and normalizing the SMS messages. This may involve removing special characters, lowercasing all the text, and splitting the messages into individual words.

- Once you have a cleaned and normalized dataset, you can start building your model. One approach you could take is to use a sequence labeling model, such as a Conditional Random Field (CRF), to identify and classify the various pieces of transaction information within the SMS messages.

- To train your model, you will need to split your dataset into a training set and a test set. You can then use the training set to fit your model and the test set to evaluate its performance.

- Finally, you can use your trained model to extract transaction information from new, unseen bank SMS messages. You can do this by passing the SMS messages through your model and using the predicted labels to extract the relevant information.

In [ ]:



import nltk

def tokenize(text):
    tokens = nltk.word_tokenize(text)
    return tokens

sms_messages = ["ATM Withdrawal Rs 30000.00 From A/C No XXXXXXXXXX400. Balance available Rs 5963.81 - Thank you for banking with BOC","ATM Withdrawal Rs 4000.00 From A/C No XXXXXXXXXX400. Balance available Rs 16969.15 - Thank you for banking with BOC","HNB SMS ALERT: PURCHASE, Debit account:0717***4696,Location:MY CHEMIST (PVT) LTD, LK,Amount(Approx.):1399.73 LKR,Av.Bal:101534.94 LKR,Date:28.12.21,Time:13:21, Hot Line:0112462462"]

sms_tokens = [tokenize(sms) for sms in sms_messages]

print(sms_tokens)